# Transform Detectron2 Mask RCNN model to TensorRT optimizer model

This notebook follows the guide in https://github.com/NVIDIA/TensorRT/tree/main/samples/python/detectron2

In [1]:
!nvidia-smi

Tue Feb 28 05:21:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    23W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.5 LTS
Release:	20.04
Codename:	focal


## Install latest version of Pytorch


* Unistall the default Pytorch Version

In [3]:
!pip uninstall torch --yes

Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116


* Install the newest one

In [4]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 866.7 kB/s eta 0:00:00


## Install Tensor RT

We need to install the following packages: nvidia-cublas-cu11-11.11.3.6 nvidia-cuda-runtime-cu11-11.8.89 nvidia-cudnn-cu11-8.8.0.121 nvidia-tensorrt-99.0.0 tensorrt-8.5.3.1

In [5]:
#!pip install nvidia-tensorrt==8.4.1.5
!pip install nvidia-tensorrt #Install the latest version of Tensor RT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.5/549.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.3/720.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 KB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.1 MB/s eta 0:00:00


## Clone NVIDIA-TENSORT transformation repository and install requirements.

Clone the repository

In [7]:
!git clone https://github.com/FabricioCrespo/Detectron2_MaskRCNN_2_TensorRT_utils.git

Cloning into 'Detectron2_MaskRCNN_2_TensorRT_utils'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 0), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 31.61 KiB | 5.27 MiB/s, done.


Installing requirements

In [8]:
!pip install -r /content/Detectron2_MaskRCNN_2_TensorRT_utils/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/FabricioCrespo/Detectron2_maskrcnn_to_tensorrt_update.git to /tmp/pip-req-build-3714bedg
  Running command git clone --filter=blob:none --quiet https://github.com/FabricioCrespo/Detectron2_maskrcnn_to_tensorrt_update.git /tmp/pip-req-build-3714bedg
  Resolved https://github.com/FabricioCrespo/Detectron2_maskrcnn_to_tensorrt_update.git to commit a868fa3974b89269cca0d978bda3facf161349f3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
!python3 -m pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 KB 4.0 MB/s eta 0:00:00


* Verify the instalation.

In [10]:
!pip show onnx_graphsurgeon
!pip show onnx
!pip show onnxruntime
!pip show torch
!pip show torchvision
!pip show torchaudio
!pip show nvidia-tensorrt
!pip show numpy
!pip show cuda-python
!pip show Pillow

Name: onnx-graphsurgeon
Version: 0.3.26
Summary: ONNX GraphSurgeon
Home-page: https://github.com/nvidia/tensorrt/tools/onnx-graphsurgeon
Author: NVIDIA
Author-email: svc_tensorrt@nvidia.com
License: Apache 2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: numpy, onnx
Required-by: 
Name: onnx
Version: 1.13.1
Summary: Open Neural Network Exchange
Home-page: https://github.com/onnx/onnx
Author: ONNX
Author-email: onnx-technical-discuss@lists.lfaidata.foundation
License: Apache License v2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: numpy, protobuf, typing-extensions
Required-by: onnx-graphsurgeon
Name: onnxruntime
Version: 1.14.1
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: /usr/local/lib/python3.8/dist-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: 
Nam

In [13]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
antlr4-python3-runtime        4.9.3
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
black 

In [15]:
!mv /content/Detectron2_MaskRCNN_2_TensorRT_utils /content/detectron2_2_tensorrt

## Clone Detectron2

In [16]:
!git clone https://github.com/FabricioCrespo/Detectron2_maskrcnn_to_tensorrt_update.git

Cloning into 'Detectron2_maskrcnn_to_tensorrt_update'...
remote: Enumerating objects: 14731, done.
remote: Counting objects: 100% (14731/14731), done.
remote: Compressing objects: 100% (3953/3953), done.
remote: Total 14731 (delta 10695), reused 14731 (delta 10695), pack-reused 0
Receiving objects: 100% (14731/14731), 5.99 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (10695/10695), done.


In [17]:
!mv /content/Detectron2_maskrcnn_to_tensorrt_update /content/detectron2

## From Detectron2 to Tensort RT

### Mandatory changes in each python code.

In order to avoid problems related to the conversion of the model, it is necessary to set the number of classes in each **python file** to be used as follows:

```
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 #1 because we have only strawberry as the unique class

```
In our particular case, the number of classes is 1 (because strawberry detection). 

Deployment is done through export model script located in detectron2/tools/deploy/export_model.py of Detectron 2 github. Detectron 2 Mask R-CNN R50-FPN 3x model is dynamic with minimum testing dimension size of 800 and maximum of 1333. TensorRT plug-ins used for conversion of this model do not support dynamic shapes, as a result we have to set both height and width of the input tensor to 1344. 1344 instead of 1333 because model requires both height and width of the input tensor to be divisible by 32. In order to export this model with correct 1344x1344 resolution, we have to make a change to export_model.py. Currently lines 165-170:

```
aug = T.ResizeShortestEdge(
    [1344, 1344], 1344
)
```

The detectron2 repository at FabricioCrespo repo already has changed these lines. So, you dont need to change the lines.

#### Copy the Detectron2 Mask RCNN model to colab.

In [18]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Copy the Detectron2 model (pth file) to the local storage

In [19]:
#!cp /content/gdrive/MyDrive/model_final_f10217.pkl .
!cp /content/gdrive/MyDrive/strawberry_desease/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth .
#!cp /content/gdrive/MyDrive/AI_PROJECTS/STRAWBERRY_MASKRCNN/pretrained_models/model_final_f10217.pkl .

**Note**: To transfor the model you should need ONNX OPSET VERSION = 16. The repo already has set the value to 16.

In [20]:
import detectron2.export

print(detectron2.export.STABLE_ONNX_OPSET_VERSION)

16


#### Export model

Go to the file `/content/detectron2/tools/deploy/export_model.py` and below line 35 add the number of classes to the `cfg`. In our particular case `cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1`

Export script takes `--sample-image` as one of the arguments. Such image is used to adjust input dimensions and dimensions of tensors for the rest of the network. This sample image has to be an image of 1344x1344 dimensions, which contains at least one detectable by model object. My recommendation is to upload a resized samble image from our strawberry dataset (size = 1344x1344).

In [21]:
!python3 detectron2/tools/deploy/export_model.py \
    --sample-image /content/resized_image.png \
    --config-file /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
    --export-method tracing \
    --format onnx \
    --output ./ \
    MODEL.WEIGHTS /content/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth \
    MODEL.DEVICE cuda


[02/28 06:17:18 detectron2]: Command line arguments: Namespace(config_file='/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml', export_method='tracing', format='onnx', opts=['MODEL.WEIGHTS', '/content/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth', 'MODEL.DEVICE', 'cuda'], output='./', run_eval=False, sample_image='/content/resized_image.png')
[W init.cpp:1043] Warning: Use _jit_set_fusion_strategy, bailout depth is deprecated. Setting to (STATIC, 1) (function operator())
[02/28 06:17:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth ...
/usr/local/lib/python3.8/dist-packages/detectron2/structures/image_list.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the tra

Where `--sample-image` is 1344x1344 image; `--config-file` path to Mask R-CNN R50-FPN 3x config, included with detectron2; `MODEL.WEIGHTS` are weights from our custom Detectron2 pth model. Resulted `model.onnx` will be an input to conversion script.

### Create ONNX Graph
This is supported Detectron 2 model:

| **Model**                                     | **Resolution** |
| ----------------------------------------------|----------------|
| Mask R-CNN R50-FPN 3x                         | 1344x1344      |

Go to `/content/detectron2_2_tensorrt/create_onnx.py` file and in line 63 put the number of classes (in our case 1 only for strawberry `cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1`).

If Detectron 2 Mask R-CNN is ready to be converted (i.e. you ran `detectron2/tools/deploy/export_model.py`), run:

In [22]:
!python /content/detectron2_2_tensorrt/create_onnx.py \
    --exported_onnx /content/model.onnx \
    --onnx /content/model.onnx \
    --det2_config /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
    --det2_weights /content/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth \
    --sample_image /content/resized_image.png \
    --batch_size 1

INFO:ModelHelper:ONNX graph loaded successfully
INFO:ModelHelper:Number of FPN output channels is 256
INFO:ModelHelper:Number of classes is 1
INFO:ModelHelper:First NMS max proposals is 1000
INFO:ModelHelper:First NMS iou threshold is 0.7
INFO:ModelHelper:First NMS score threshold is 0.01
INFO:ModelHelper:First ROIAlign type is ROIAlignV2
INFO:ModelHelper:First ROIAlign pooled size is 7
INFO:ModelHelper:First ROIAlign sampling ratio is 0
INFO:ModelHelper:Second NMS max proposals is 100
INFO:ModelHelper:Second NMS iou threshold is 0.5
INFO:ModelHelper:Second NMS score threshold is 0.05
INFO:ModelHelper:Second ROIAlign type is ROIAlignV2
INFO:ModelHelper:Second ROIAlign pooled size is 14
INFO:ModelHelper:Second ROIAlign sampling ratio is 0
INFO:ModelHelper:Individual mask output resolution is 28x28
2023-02-28 06:42:05.911409659 [W:onnxruntime:, unsqueeze_elimination.cc:20 Apply] UnsqueezeElimination cannot remove node /roi_heads/Unsqueeze_9
2023-02-28 06:42:05.911446814 [W:onnxruntime:, 

This will create the file `converted.onnx` which is ready to convert to TensorRT.

It is important to mention that `--sample_image` in this case is used for anchor generation. Detectron 2 ONNX models do not have anchor data inside the graph, so anchors have to be generated "offline". If custom model is used, make sure preprocessing of your model matches what is coded in `get_anchors(self, sample_image)` function.

The script has a few optional arguments, including:

* `--first_nms_threshold [...]` allows overriding the default 1st NMS score threshold parameter, as the runtime latency of the NMS plugin is sensitive to this value. It's a good practice to set this value as high as possible, while still fulfilling your application requirements, to reduce inference latency. In Mask R-CNN this will be a score threshold for Region Proposal Network.
* `--second_nms_threshold [...]` allows overriding the default 2nd NMS score threshold parameter, further improves the runtime latency of the NMS plugin. It's a good practice to set this value as high as possible, while still fulfilling your application requirements, to reduce inference latency. It will be the second and last NMS.
* `--batch_size` allows selection of various batch sizes, default is 1.


Optionally, you may wish to visualize the resulting ONNX graph with a tool such as [Netron](https://netron.app/).

The input to the graph is a `float32` tensor with the selected input shape, containing RGB pixel data in the range of 0 to 255. All preprocessing will be performed inside the Model graph, so it is not required to further pre-process the input data.


The outputs of the graph are the same as the outputs of the [EfficientNMS_TRT](https://github.com/NVIDIA/TensorRT/tree/master/plugin/efficientNMSPlugin) plugin and segmentation head output, name of the last node is `detection_masks`, shape is `[batch_size, max_proposals, mask_height, mask_width]`, dtype is float32.

### Build TensorRT Engine

TensorRT engine can be built directly with `trtexec` using the ONNX graph generated in the previous step. If it's not already in your `$PATH`, the `trtexec` binary is usually found in `/usr/src/tensorrt/bin/trtexec`, depending on your TensorRT installation method. Run:

```
trtexec --onnx=/path/to/converted.onnx --saveEngine=/path/to/engine.trt --useCudaGraph
```

However, the script `build_engine.py` is also provided in this repository for convenience, as it has been tailored to Detectron 2 2 Mask R-CNN R50-FPN 3x engine building and INT8 calibration. Run `python3 build_engine.py --help` for details on available settings.



#### FP32 Precision

To build the TensorRT engine file with FP32 precision, run:

In [ ]:
%cd /content/

!python /content/detectron2_2_tensorrt/build_engine.py \
    --onnx /content/model.onnx \
    --engine ./enginFP32.trt \
    --precision fp32 

/content
[02/28/2023-03:02:58] [TRT] [I] [MemUsageChange] Init CUDA: CPU +308, GPU +0, now: CPU 394, GPU 237 (MiB)
[02/28/2023-03:03:01] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +262, GPU +76, now: CPU 711, GPU 313 (MiB)
[02/28/2023-03:03:01] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
/content/detectron2_2_tensorrt/build_engine.py:147: DeprecationWarning: Use set_memory_pool_limit instead.
  self.config.max_workspace_size = workspace * (2 ** 30)
[02/28/2023-03:03:01] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[02/28/2023-03:03:01] [TRT] [I] No importer registered for op: EfficientNMS_TRT. Attempting to import as plugin.
[02/28/2023-03:03:01] [TRT] [I] Searching for plugin: Efficient

#### FP16 Precision

To build the TensorRT engine file with FP16 precision, run:

In [23]:
%cd /content/

!python /content/detectron2_2_tensorrt/build_engine.py \
    --onnx /content/model.onnx \
    --engine ./engineFP16.trt \
    --precision fp16 

/content
[02/28/2023-06:43:55] [TRT] [I] [MemUsageChange] Init CUDA: CPU +308, GPU +0, now: CPU 394, GPU 237 (MiB)
[02/28/2023-06:43:59] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +262, GPU +76, now: CPU 711, GPU 313 (MiB)
[02/28/2023-06:43:59] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
/content/detectron2_2_tensorrt/build_engine.py:147: DeprecationWarning: Use set_memory_pool_limit instead.
  self.config.max_workspace_size = workspace * (2 ** 30)
[02/28/2023-06:44:00] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[02/28/2023-06:44:00] [TRT] [I] No importer registered for op: EfficientNMS_TRT. Attempting to import as plugin.
[02/28/2023-06:44:00] [TRT] [I] Searching for plugin: Efficient

### Import complete dataset to make calibration.

Where `--calib_input` points to a directory with several thousands of images. For example, this could be a subset of the training or validation datasets that were used for the model. It's important that this data represents the runtime data distribution relatively well, therefore, the more images that are used for calibration, the better accuracy that will be achieved in INT8 precision. For models trained for the [COCO dataset](https://cocodataset.org/#home), we have found that 5,000 images gives a good result.

In [ ]:
%cd /content/
!cp /content/gdrive/MyDrive/strawberry_desease/StrawDI_Db1.zip .
!unzip /content/StrawDI_Db1.zip
!mv StrawDI_Db1 dataset

Streaming output truncated to the last 5000 lines.
  inflating: StrawDI_Db1/train/img/180.png  
  inflating: StrawDI_Db1/train/img/1800.png  
  inflating: StrawDI_Db1/train/img/1801.png  
  inflating: StrawDI_Db1/train/img/1802.png  
  inflating: StrawDI_Db1/train/img/1804.png  
  inflating: StrawDI_Db1/train/img/1805.png  
  inflating: StrawDI_Db1/train/img/1806.png  
  inflating: StrawDI_Db1/train/img/1807.png  
  inflating: StrawDI_Db1/train/img/1808.png  
  inflating: StrawDI_Db1/train/img/1809.png  
  inflating: StrawDI_Db1/train/img/181.png  
  inflating: StrawDI_Db1/train/img/1810.png  
  inflating: StrawDI_Db1/train/img/1811.png  
  inflating: StrawDI_Db1/train/img/1812.png  
  inflating: StrawDI_Db1/train/img/1814.png  
  inflating: StrawDI_Db1/train/img/1815.png  
  inflating: StrawDI_Db1/train/img/1816.png  
  inflating: StrawDI_Db1/train/img/1817.png  
  inflating: StrawDI_Db1/train/img/1818.png  
  inflating: StrawDI_Db1/train/img/1819.png  
  inflating: StrawDI_Db1/train/

#### INT8 Precision

To build the TensorRT engine file with INT8 precision, run:

In [ ]:
%cd /content/

!python /content/detectron2_2_tensorrt/build_engine.py \
    --onnx /content/model.onnx \
    --engine ./engineINT8.trt \
    --det2_config /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
    --precision int8 \
    --calib_input /content/dataset/train/img \
    --calib_cache /content/calibration.cache

/content
[02/28/2023-03:45:04] [TRT] [I] [MemUsageChange] Init CUDA: CPU +308, GPU +0, now: CPU 394, GPU 237 (MiB)
[02/28/2023-03:45:11] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +262, GPU +76, now: CPU 711, GPU 313 (MiB)
[02/28/2023-03:45:11] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
/content/detectron2_2_tensorrt/build_engine.py:147: DeprecationWarning: Use set_memory_pool_limit instead.
  self.config.max_workspace_size = workspace * (2 ** 30)
[02/28/2023-03:45:11] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[02/28/2023-03:45:11] [TRT] [I] No importer registered for op: EfficientNMS_TRT. Attempting to import as plugin.
[02/28/2023-03:45:11] [TRT] [I] Searching for plugin: Efficient

The file `engine.trt` will be created, which can now be used to infer with TensorRT.

For best results, make sure no other processes are using the GPU during engine build, as it may affect the optimal tactic selection process.

### Inference in Python







In [24]:
%cd /content/
!mkdir images
!mkdir output

/content


First, you need to create `images` and `output` folders. In `images` folder you can upload your images and in the `output` folder you will get the results.
To perform object detection on a set of images with TensorRT, run:

In [25]:
!python /content/detectron2_2_tensorrt/infer.py \
    --engine /content/engineFP16.trt \
    --input /content/images \
    --det2_config /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
    --output /content/output \
    --nms_threshold 0.55

/content/detectron2_2_tensorrt/infer.py:54: DeprecationWarning: Use get_tensor_mode instead.
  if self.engine.binding_is_input(i):
/content/detectron2_2_tensorrt/infer.py:56: DeprecationWarning: Use get_tensor_name instead.
  name = self.engine.get_binding_name(i)
/content/detectron2_2_tensorrt/infer.py:57: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = self.engine.get_binding_dtype(i)
/content/detectron2_2_tensorrt/infer.py:58: DeprecationWarning: Use get_tensor_shape instead.
  shape = self.engine.get_binding_shape(i)
/content/detectron2_2_tensorrt/infer.py:74: DeprecationWarning: Use get_tensor_mode instead.
  if self.engine.binding_is_input(i):

Finished Processing


Where the input path can be either a single image file, or a directory of jpg/png/bmp images.

The script has a few optional arguments, including:
* `--nms_threshold` allows overriding the default second NMS score threshold parameter.
* `--iou_threshold` allows to set IoU threshold for the mask segmentation, default is 0.5.

The detection results will be written out to the specified output directory, consisting of a visualization image, and a tab-separated results file for each input image processed.

### Evaluate mAP Metric (FALTA COMPLETAR EL CODIGO Y COMPROBAR)



#### Import Dataset from dive

In [26]:
!cp /content/gdrive/MyDrive/strawberry_desease/coco.zip /content/

In [27]:
!unzip /content/coco.zip -d .

Archive:  /content/coco.zip
   creating: ./coco/
   creating: ./coco/val2017/
  inflating: ./coco/val2017/1401.png  
  inflating: ./coco/val2017/299.png  
  inflating: ./coco/val2017/1784.png  
  inflating: ./coco/val2017/699.png  
  inflating: ./coco/val2017/1218.png  
  inflating: ./coco/val2017/514.png  
  inflating: ./coco/val2017/952.png  
  inflating: ./coco/val2017/942.png  
  inflating: ./coco/val2017/795.png  
  inflating: ./coco/val2017/470.png  
  inflating: ./coco/val2017/1941.png  
  inflating: ./coco/val2017/980.png  
  inflating: ./coco/val2017/809.png  
  inflating: ./coco/val2017/1907.png  
  inflating: ./coco/val2017/2913.png  
  inflating: ./coco/val2017/2304.png  
  inflating: ./coco/val2017/1669.png  
  inflating: ./coco/val2017/443.png  
  inflating: ./coco/val2017/670.png  
  inflating: ./coco/val2017/2904.png  
  inflating: ./coco/val2017/1355.png  
  inflating: ./coco/val2017/1395.png  
  inflating: ./coco/val2017/2807.png  
  inflating: ./coco/val2017/127.png 

In [28]:
!mkdir /content/detectron2_2_tensorrt/datasets

In [29]:
!mv /content/coco /content/detectron2_2_tensorrt/datasets

Given a validation dataset (such as [COCO val2017 data](http://images.cocodataset.org/zips/val2017.zip)), you can get the mAP metrics for the built TensorRT engine. This will use the mAP metrics tools functions from the [Detectron 2 evaluation](https://github.com/facebookresearch/detectron2/tree/main/detectron2/evaluation) repository. Make sure you follow [Use Builtin Datasets guide](https://detectron2.readthedocs.io/en/latest/tutorials/builtin_datasets.html) to correctly setup COCO or custom dataset. Additionally, run `eval_coco.py` in the same folder where `/datasets` is present, otherwise this error will appear:

```
FileNotFoundError: [Errno 2] No such file or directory: 'datasets/coco/annotations/instances_val2017.json'
```

To run evalutions, run:

In [30]:
%cd /content/detectron2_2_tensorrt

/content/detectron2_2_tensorrt


**Note**: Change the tensor rt `--engine` model name in order to evaluate the desired model

In [31]:
!python /content/detectron2_2_tensorrt/eval_coco.py \
    --engine /content/engineFP16.trt \
    --input /content/detectron2_2_tensorrt/datasets/coco/val2017 \
    --det2_config /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
    --det2_weights /content/strawberry_resnet50_AP83320_20230206_4K_iters_iter3200_lr1e-5.pth

INFO:detectron2.evaluation.coco_evaluation:Preparing results for COCO format ...
INFO:detectron2.evaluation.coco_evaluation:Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
INFO:detectron2.evaluation.fast_eval_api:Evaluate annotation type *bbox*
INFO:detectron2.evaluation.fast_eval_api:COCOeval_opt.evaluate() finished in 0.03 seconds.
INFO:detectron2.evaluation.fast_eval_api:Accumulating evaluation results...
INFO:detectron2.evaluation.fast_eval_api:COCOeval_opt.accumulate() finished in 0.01 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.815
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.968
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.884
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.508
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.858
 Average Pr

The script has a few optional arguments, including:
* `--nms_threshold` allows overriding the default second NMS score threshold parameter.
* `--iou_threshold` allows to set IoU threshold for the mask segmentation, default is 0.5.

The mAP metric is sensitive to the NMS score threshold used, as using a high threshold will reduce the model recall, resulting in a lower mAP value. It may be a good idea to build separate TensorRT engines for different purposes. That is, one engine with a default threshold (like 0.5) dedicated for mAP validation, and another engine with your application specific threshold (like 0.8) for deployment. This is why we keep the NMS threshold as a configurable parameter in the `create_onnx.py` script.

### COPY MODELS TO DRIVE

In [ ]:
!cp /content/enginFP32.trt /content/gdrive/MyDrive/strawberry_desease
!cp /content/engineFP16.trt /content/gdrive/MyDrive/strawberry_desease
!cp /content/engineINT8.trt /content/gdrive/MyDrive/strawberry_desease
!cp /content/calibration.cache /content/gdrive/MyDrive/strawberry_desease